#### Send ontology file for importation

In [23]:
import requests

app_api_host = 'https://api.app.lettria.com'
file_name = 'ontology.owl'
file_location = './data/'
api_route = app_api_host + '/app/knowledge/import'
jwt_token_file = 'jwt_token.txt'
jwt_token = open(jwt_token_file, 'r').read()
knowlede_id = None

with open(file_location + file_name, 'rb') as file:
    response = requests.post(
        api_route,
        files=[('file', ( file_name, file, "application/rdf+xml"))],
        headers={"Authorization":"LettriaProKey " + jwt_token}
    )
    print(response.json())
    knowlede_id = response.json()['knowledgeId']

{'message': 'ok', 'knowledgeId': '6573407f49582e002a8bb7fb'}


#### Polling to wait for the end of the import

In [24]:
import time
import requests

api_route = app_api_host + '/app/knowledge/read'
importation_is_finished = False

knowledge = None
importation_status = None
while not importation_is_finished:
    response = requests.post(
        api_route,
        headers={"Authorization":"LettriaProKey " + jwt_token}
    ).json()
    imported_knowledge = next((item for item in response['documents'] if item["_id"] == knowlede_id), None)
    importation_status = imported_knowledge.get('importationStatus')
    print(importation_status + "      ", end='\r')
    importation_is_finished = importation_status != 'PROCESSING'
    time.sleep(1)
if importation_status != 'SUCCESS':
    print('Importation failed')
    exit(1)

#### Extract document text chunks

In [25]:
api_host = 'https://api.lettria.com'
api_route = api_host + '/parse'
document_file_name = 'file.txt'
document_file_location = './data/'

chunks = []
with open(document_file_location + document_file_name, 'rb') as file:
    response = requests.post(
        api_route,
        files=[('file', ( document_file_name, file, "application/text"))],
        headers={"Authorization":"LettriaProKey " + jwt_token}
    ).json()
    text_chunks = response['chunks']

text_chunks = [chunk['content'] for chunk in text_chunks if chunk['type'] == 'text']
print(text_chunks)

['Mon cheval de 30kg']


#### Text to graph using Lettria structuration endpoint

In [26]:
api_route = api_host + '/structuration'

response = requests.post(
    api_route,
    json={
        "documents": text_chunks,
        "options":{
            "rdf_export": True
        }
    },
    headers={"Authorization":"LettriaProKey " + jwt_token}
).json()

rdf_string = response['rdf']
print(rdf_string)

@prefix card: <https://data.lettria.com/card/> .
@prefix d: <https://doc.lettria.com/ontology/1.0/Data/> .
@prefix info: <https://data.lettria.com/info/> .
@prefix l: <https://doc.lettria.com/ontology/1.0/> .
@prefix m: <https://doc.lettria.com/ontology/1.0/Metadata/> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<https://data.lettria.com/project/63f642b44aaf52003163cecc> a l:Project ;
    rdfs:label "63f642b44aaf52003163cecc" ;
    l:Document <https://data.lettria.com/document/63f642b44aaf52003163cecc_document_0> ;
    m:platformUrl "https://app.lettria.com/nlp/project/view/63f642b44aaf52003163cecc/overview" ;
    m:projectId "63f642b44aaf52003163cecc" .

card:b7e82813-e509-4c02-82ce-bfcbeccb671e a owl:Class,
        l:Card ;
    d:categopath "concrete_thing|natural_thing|person" ;
    d:categosens "concrete_thing"